In [1]:
import sys
sys.path.append('../')

In [2]:
from scripts.utils import data_path
from scripts.cross_match_scripts import my_scaler_forward
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_columns', 500)
import seaborn as sns
import numpy as np
%matplotlib inline

matplotlib settings set


matplotlib settings set


# Catalogs loading and preprocessing

In [3]:
photo_cat = pd.read_pickle(data_path+'csc_allsky_desi_r30_gaia_dered_training.gz_pkl', compression='gzip')
x_ray_flux_lim = 3e-15
photo_cat = photo_cat.query('(csc_flux_05_2 > @x_ray_flux_lim) | is_counterpart == False')
photo_cat = photo_cat.replace([np.inf, -np.inf], np.nan)
photo_cat.sample(5)

,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_radec_err,csc_r_98,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,r_false_003_dens_10_30,num_srcs_sep_less_r_false,is_counterpart
249020,2CXO J020931.7-000448,32.382143,-0.080078,NaN,1.533755,NaN,9010,4976,330497,32.388144,-0.085282,1,0.100943,0.197464,0.670935,1.408637,0.558947,11.596489,-286.37520,2914.48050,1875.35700,243.111270,3.343958,0.699682,0.001385,0.000019,24.901377,24.201708,22.899979,22.122940,23.128464,19.838520,NaN,5.449494,8.551265,10.461229,2.575903,0.467542,0.431596,-1.243862,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,28.597403,9010_330497_4976,24.989809,24.989809,24.901377,24.261280,24.261280,24.201708,22.933299,22.933299,22.899979,22.128002,NaN,NaN,NaN,23.131573,NaN,NaN,NaN,19.839184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.155035e-15,0.699669,2.001398,1.301729,NaN,NaN,NaN,NaN,NaN,NaN,0.846574,1,False
764497,2CXO J100059.4+013232,150.247804,1.542346,NaN,0.463603,NaN,9010,5170,339608,150.248465,1.546903,1,0.135034,0.205974,0.339744,1.254593,-1.149354,-110.014946,312.61023,1262.98690,462.62613,80.723885,3.164188,0.668670,0.000767,0.000011,24.607716,24.170890,23.647186,22.249954,NaN,NaN,16.262304,4.798902,4.430245,3.052479,2.231691,-0.939853,-3.047724,1.030404,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,16.578355,9010_339608_5170,24.673892,24.673892,24.607716,24.215469,24.215469,24.170890,23.672121,NaN,NaN,22.253743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.262492,NaN,NaN,NaN,NaN,0.436826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.052429,1,False
406043,2CXO J033847.6-354036,54.698639,-35.676700,NaN,0.521755,NaN,9010,748,137594,54.694899,-35.674149,1,0.051346,0.089996,0.182935,-0.988696,0.759423,-5.243128,-725.60280,4250.96200,2840.29930,247.476960,4.237062,1.052784,0.003643,0.000044,25.686483,25.089355,24.330225,NaN,22.797480,NaN,NaN,3.347747,4.796278,2.877825,-2.035143,0.779208,-0.316446,-4.808335,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,14.282257,9010_137594_748,25.723733,NaN,NaN,25.114442,25.114442,25.089355,24.344258,NaN,NaN,NaN,NaN,NaN,NaN,22.798791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.790446,1,False
1446025,2CXO J180726.7+694627,271.861395,69.774248,8.931019e-15,0.554394,1.550723,9011,516,641589,271.860800,69.774225,1,14.108253,15.066680,17.604320,13.222942,17.236004,99.724870,305.83040,163.85246,157.26503,97.413864,8.735271,3.535283,0.013068,0.000152,19.503834,19.472450,19.339802,19.689669,19.404602,17.502073,16.285952,180.592500,188.944230,173.751940,39.081060,32.407738,11.399920,3.769119,0.215419,9.75616,0.955395,2.973269,1.418538,2.018895,0.744794,9011_641589_516,19.626317,19.626317,19.503834,19.554956,19.554956,19.472450,19.385952,19.385952,19.339802,19.696680,19.69668,19.689669,16.99768,19.408909,19.408909,19.404602,16.069909,17.502991,17.502991,17.502073,12.328991,16.286298,NaN,NaN,NaN,3.066664e-14,0.031384,0.164032,0.132648,-0.349867,0.067848,0.285067,1.837729,NaN,NaN,1.103797,1,True
1348673,2CXO J155706.4+060528,239.276796,6.091150,NaN,2.007730,NaN,9010,594,365843,239.271981,6.089057,1,0.125563,0.289781,0.894671,1.776213,-1.280163,-71.143166,-807.72300,914.24880,406.40800,48.722050,3.659949,0

In [4]:
print('fraction of missed measurements per is_counterpart per filter, in per cent:')
photo_cat.groupby('is_counterpart')[['rel_dered_mag_g',	'rel_dered_mag_r',	'rel_dered_mag_z',	'rel_dered_mag_w1',	'rel_dered_mag_w2',	'rel_dered_mag_w3',	'rel_dered_mag_w4']].agg(lambda x: np.round(100*np.mean(np.
isnan(x)),1))

fraction of missed measurements per is_counterpart per filter, in per cent:


,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_mag_w3,rel_dered_mag_w4
is_counterpart,,,,,,,
False,21.1,14.0,24.4,59.5,76.2,95.6,85.8
True,7.4,4.5,2.0,5.0,9.2,70.7,79.4


In [5]:
features_cols = ['rel_dered_mag_g','rel_dered_mag_r','rel_dered_mag_z'] 
features_cols += ['rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z']


features_cols += ['rel_dered_mag_w1', 'rel_dered_mag_w2']
features_cols += ['rel_dered_z_w1', 'rel_dered_r_w2', 'rel_dered_w1_w2']

features_cols += ['rel_dered_mag_w3', 'rel_dered_mag_w4']
features_cols += ['rel_dered_z_w3', 'rel_dered_r_w4', 'rel_dered_w3_w4']


target_col = ['is_counterpart']

photo_cat = photo_cat[features_cols+target_col+['csc_flux_05_2']]

#photo_cat= photo_cat.dropna(subset = features_cols, how = 'any')

print(photo_cat.is_counterpart.value_counts())
photo_cat = photo_cat.sample(frac=1)
photo_cat.reset_index(inplace=True, drop = True)

photo_cat_scaled = my_scaler_forward(photo_cat)
photo_cat_scaled.to_pickle('./model/train_test_validation_data.pkl')
photo_cat_scaled.head(15)


False    368740
True      35806
Name: is_counterpart, dtype: int64
rel_dered_mag_g scaled by 1/35
rel_dered_mag_r scaled by 1/35
rel_dered_mag_z scaled by 1/35
rel_dered_g_r scaled by 1/10
rel_dered_r_z scaled by 1/10
rel_dered_g_z scaled by 1/10
rel_dered_mag_w1 scaled by 1/35
rel_dered_mag_w2 scaled by 1/35
rel_dered_z_w1 scaled by 1/10
rel_dered_r_w2 scaled by 1/10
rel_dered_w1_w2 scaled by 1/10
rel_dered_mag_w3 scaled by 1/35
rel_dered_mag_w4 scaled by 1/35
rel_dered_z_w3 scaled by 1/10
rel_dered_r_w4 scaled by 1/10
rel_dered_w3_w4 scaled by 1/10


,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_g_r,rel_dered_r_z,rel_dered_g_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_mag_w3,rel_dered_mag_w4,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,is_counterpart,csc_flux_05_2
0,0.691683,0.681138,0.668626,0.036908,0.043792,0.080699,NaN,NaN,NaN,NaN,NaN,NaN,0.375012,NaN,1.071440,NaN,False,NaN
1,NaN,0.686543,0.645466,NaN,0.143771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.428864,NaN,0.901876,NaN,False,NaN
2,0.722051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
3,0.708018,0.697223,NaN,0.037783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
4,0.677138,0.671650,0.649900,0.019208,0.076124,0.095332,NaN,NaN,NaN,NaN,NaN,0.481042,NaN,0.591005,NaN,NaN,False,NaN
5,0.685361,0.671447,NaN,0.048697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.393927,NaN,0.971321,NaN,False,NaN
6,0.695812,0.662285,0.620864,0.117345,0.144973,0.262317,0.575200,0.576799,0.159824,0.299202,-0.005595,NaN,0.417976,NaN,0.855080,NaN,False,NaN
7,0.323730,0.330203,0.333110,-0.022657,-0.010172,-0.032829,0.368868,0.386736,-0.125155,-0.197865,-0.062538,0.431148,NaN,-0.343135,NaN,NaN,True,2.506598e-14
8,0.667277,0.617225,0.591876,0.175180,0.088724,0.263905,0.578999,NaN,0.045066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
9,NaN,NaN,0.646403,NaN,NaN,NaN,0.564106,0.554851,0.288042,NaN,0.032392,NaN,NaN,NaN,NaN,NaN,False,NaN
